# Cyclic Groups and Generators

**Module 01d** | Modular Arithmetic and Groups

*Some elements can regenerate the entire group by themselves. These elements are the foundation of modern cryptography.*

> **Question:** In $(\mathbb{Z}/7\mathbb{Z}^*, \times)$, start with 3 and compute its powers: $3, 3^2, 3^3, 3^4, 3^5, 3^6$. You get $\{3, 2, 6, 4, 5, 1\}$ — the ENTIRE group from one element! But start with 2: $2, 2^2, 2^3 = \{2, 4, 1\}$. Only half the group. Why does 3 "reach everywhere" and 2 doesn't?

## Objectives

By the end of this notebook you will be able to:

1. Compute the **order** of an element in a group
2. Determine whether an element is a **generator**
3. Count generators using **Euler's totient function**
4. Connect generators to the **discrete logarithm problem**

## From Groups to Generators

In [01c](01c-groups-first-look.ipynb) we learned that $(\mathbb{Z}/7\mathbb{Z}^*, \times)$ is a group — you can multiply and find inverses freely. But some elements are more *powerful* than others: they can regenerate the entire group by themselves. These are **generators**, and they're the cornerstone of Diffie-Hellman, ElGamal, and DSA.

## Powers Trace Out a Path

Let's compute the successive powers of every element in $(\mathbb{Z}/7\mathbb{Z}^*, \times)$.

In [ ]:
# Compute g^1, g^2, ..., g^6 for every element g in (Z/7Z)*
p = 7
print(f"Powers of each element in (Z/{p}Z)*")
print("=" * 55)
print(f"{'g':>3} | {'powers g^1, g^2, ..., g^6':^34} | {'# distinct':>10}")
print("-" * 55)

for g in range(1, p):
    powers = [power_mod(g, k, p) for k in range(1, p)]
    distinct = len(set(powers))
    label = "<-- FULL GROUP" if distinct == p - 1 else ""
    print(f"{g:>3} | {str(powers):^34} | {distinct:>10}  {label}")

> **Checkpoint:** Look at the orbit of 6: it's $\{6, 1\}$ — just 2 elements. Can you predict this orbit length *before computing*? (Hint: $6 \equiv -1 \pmod{7}$, so $6^2 = (-1)^2 = 1$.)

## Order of an Element

The **order** of $g$, written $\text{ord}(g)$, is the smallest positive $k$ such that $g^k = 1$. SageMath computes this with `multiplicative_order()`.

In [ ]:
# Order table for all elements of (Z/7Z)*
p = 7
group_order = p - 1  # |G| = phi(p) = p-1 for prime p

print(f"Orders of elements in (Z/{p}Z)*    [|G| = {group_order}]")
print("=" * 45)
print(f"{'element':>8} | {'ord(g)':>8} | {'generator?':>12}")
print("-" * 45)

generator_count = 0
for g in range(1, p):
    order = Mod(g, p).multiplicative_order()
    is_gen = order == group_order
    if is_gen:
        generator_count += 1
    tag = "YES" if is_gen else ""
    print(f"{g:>8} | {order:>8} | {tag:>12}")

print("-" * 45)
print(f"Generators found: {generator_count}")
print(f"Note: every order divides |G| = {group_order}.  Divisors of {group_order}: {divisors(group_order)}")

> **Checkpoint:** Predict the possible orders in $(\mathbb{Z}/11\mathbb{Z}^*)$ before computing. The group has $11 - 1 = 10$ elements. What numbers divide 10?

In [ ]:
# Verify your predictions: orders in (Z/11Z)*
p = 11
group_order = p - 1
print(f"Divisors of {group_order}: {divisors(group_order)}")
print(f"So the only possible orders are: {divisors(group_order)}")
print()

print(f"{'element':>8} | {'ord(g)':>8} | {'generator?':>12}")
print("-" * 40)

observed_orders = set()
for g in range(1, p):
    order = Mod(g, p).multiplicative_order()
    observed_orders.add(order)
    tag = "YES" if order == group_order else ""
    print(f"{g:>8} | {order:>8} | {tag:>12}")

print()
print(f"Observed orders: {sorted(observed_orders)}")
print(f"All divide {group_order}? {all(group_order % d == 0 for d in observed_orders)}")

## Why Generators Matter for Cryptography

If $g$ generates $G$, then every element can be written as $g^k$ for a unique $k$. **Finding $k$ given $g^k$ is the Discrete Logarithm Problem (DLP)** — the hard problem behind most of modern cryptography.

In [ ]:
# Demo: solve 3^x = 5 (mod 7) by brute force
p = 7
g = 3
target = 5

print(f"Discrete logarithm: find x such that {g}^x ≡ {target} (mod {p})")
print(f"Brute-force search over all x in {{1, ..., {p-1}}}:")
print()

for x in range(1, p):
    val = power_mod(g, x, p)
    marker = "  <-- FOUND IT!" if val == target else ""
    print(f"  {g}^{x} ≡ {val} (mod {p}){marker}")

# SageMath can also compute this directly
x_solution = discrete_log(Mod(target, p), Mod(g, p))
print(f"\nSageMath confirms: log_{g}({target}) = {x_solution}  in (Z/{p}Z)*")
print(f"\nThis was trivial for |G| = {p-1}.")
print(f"For |G| ≈ 2^256, brute force would take longer than the age of the universe.")
print(f"That computational gap is what makes Diffie-Hellman, ElGamal, and DSA secure.")

## Counting Generators

There are exactly $\varphi(p-1)$ generators of $(\mathbb{Z}/p\mathbb{Z}^*)$. Let's verify.

In [ ]:
# Verify: number of generators = phi(p-1) for several primes
primes = [7, 11, 13, 23]

print(f"{'p':>5} | {'|G|=p-1':>8} | {'phi(p-1)':>9} | {'# generators':>13} | {'match?':>7}")
print("=" * 55)

for p in primes:
    group_order = p - 1
    phi_val = euler_phi(group_order)

    # Find all generators by checking orders
    generators = [g for g in range(1, p) if Mod(g, p).multiplicative_order() == group_order]
    count = len(generators)

    match = "YES" if count == phi_val else "NO"
    print(f"{p:>5} | {group_order:>8} | {phi_val:>9} | {count:>13} | {match:>7}")
    print(f"        generators: {generators}")
    print()

> **Common mistake:** "The order of an element is its numerical value." **No!** In $(\mathbb{Z}/7\mathbb{Z}^*, \times)$, element 2 has order 3, and element 3 has order 6. The order is the length of the cycle when you keep multiplying — it's a *structural* property, not a numerical one.

## Exercises

### Exercise 1 (Worked)
List all powers of 2 in $(\mathbb{Z}/13\mathbb{Z}^*)$. Determine $\text{ord}(2)$. Is 2 a generator?

In [ ]:
# Exercise 1 — Worked solution
p = 13
g = 2
group_order = p - 1  # 12

print(f"Powers of {g} in (Z/{p}Z)*:")
print()

powers = []
for k in range(1, group_order + 1):
    val = power_mod(g, k, p)
    powers.append(val)
    print(f"  {g}^{k:>2} ≡ {val:>2} (mod {p})")

order = Mod(g, p).multiplicative_order()
print(f"\nord({g}) = {order}")
print(f"|G| = {group_order}")
print(f"\nIs {g} a generator? ord({g}) == |G|?  {order} == {group_order}?  {order == group_order}")
print(f"\nYES — 2 generates all of (Z/{p}Z)* because its order equals the group order.")
print(f"The {group_order} powers produce {len(set(powers))} distinct values: {sorted(set(powers))}")

### Exercise 2 (Guided)
Find all generators of $(\mathbb{Z}/11\mathbb{Z}^*)$. Verify the count matches $\varphi(10) = 4$.

In [ ]:
# Exercise 2 — Find generators of Z/11Z*
R = Zmod(11)
group_order = 10  # = 11 - 1

# Compute orders of all elements
for x in range(1, 11):
    # TODO: compute multiplicative_order() of Mod(x, 11)
    # TODO: mark it as "GENERATOR" if its order equals group_order
    pass

# TODO: verify the count matches euler_phi(10)

### Exercise 3 (Independent)
In $(\mathbb{Z}/31\mathbb{Z}^*)$, how many generators exist? Find them all. Then pick one generator $g$ and verify that every element in $\{1, 2, \ldots, 30\}$ can be written as $g^k$ for exactly one $k \in \{1, \ldots, 30\}$. What is $\log_g(17)$?

In [ ]:
# Exercise 3 — Your code here


## Summary

- The **order** of $g$ is the smallest $k$ with $g^k = 1$; it always divides $|G|$
- $g$ is a **generator** iff $\text{ord}(g) = |G|$; then every element $= g^k$ for unique $k$
- There are exactly $\varphi(|G|)$ generators
- Finding $k$ from $g^k$ is the **discrete logarithm problem** — computationally hard for large groups

> **Crypto teaser:** In Diffie-Hellman, Alice and Bob agree on a generator $g$ of a huge group. Alice picks secret $a$, sends $g^a$. Bob picks secret $b$, sends $g^b$. They both compute $g^{ab}$ — but an eavesdropper can't, because discrete log is hard.

**Next:** [Subgroups and Lagrange's Theorem](01e-subgroups-lagrange.ipynb) — non-generators create smaller groups *inside* the big one, and their sizes are brutally constrained.